In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import platform
from matplotlib import font_manager,rc
if platform.system() == 'Windows':
# 윈도우인 경우 맑은 고딕 폰트 이용
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font',family=font_name, size=8)
else:
# Mac 인 경우
    rc('font',family='AppleGothic',size=8)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import folium,json
import folium
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import json
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [2]:
data = pd.read_csv('data/한국전력공사_법정동별_상계거래_사용전력량_20191231.csv',encoding='cp949')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56105 entries, 0 to 56104
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도      56105 non-null  object
 1   시군구     55857 non-null  object
 2   법정동     56105 non-null  object
 3   년도      56105 non-null  int64 
 4   월       56105 non-null  int64 
 5   전체호수    56105 non-null  int64 
 6   전력사용량   56105 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.0+ MB


In [3]:
gwangju = data[data['시도'] == '광주광역시']
#데이터 안에 있는 '시도' 행 에서 이름이 광주광역시인 데이터를 gwangju 변수에 저장
gwangju_rhkdtksrn = gwangju[gwangju['시군구'] == '광산구']
#위에서 저장한 gwangju 데이터 안에서 '시군구' 행에서 광산구 데이터를 gwangju_rhkdtksrn 변수에 저장
gwangju_qnrrn = gwangju[gwangju['시군구'] == '북구']
gwangju_skarn = gwangju[gwangju['시군구'] == '남구']
gwangju_ehdrn = gwangju[gwangju['시군구'] == '동구']
gwangju_tjrn = gwangju[gwangju['시군구'] == '서구']

gwangju_g1 = gwangju.groupby('시군구')['전력사용량'].sum().sort_values(ascending=False)
gwangju_g = gwangju.groupby('시군구')['전력사용량'].sum().sort_values(ascending=False)
# gwangju 안에 있는 행 '시군구' 와 '전력사용량'을 groupby 로 묶은뒤 mean() 으로 평균을 구한 후
# sort_values(ascending=Flase) 를 이용하여 평균값을 내림차순으로 정렬

In [4]:
gwangju.head(2)

시도  시군구  법정동    년도  월  전체호수  전력사용량
24324  광주광역시  광산구  고룡동  2019  1     0      0
24325  광주광역시  광산구  고룡동  2019  2     0      0

In [5]:
temp_1 = list(gwangju['전체호수'])
temp_2 = list(gwangju['전력사용량'])
plus = []
for y in range(2304):
    if temp_1[y] and temp_2[y] != 0:
        plus.append(temp_2[y]/temp_1[y])
    else:
        plus.append(0)

In [6]:
len(plus)

2304

In [7]:
gwangju['호수당전력사용량'] = plus

In [8]:
gwangju_ggh = gwangju.groupby('시군구')['호수당전력사용량'].mean()


In [9]:
gwangju_ggh

시군구
광산구    505.535920
남구     422.236345
동구     388.849938
북구     601.019233
서구     676.803126
Name: 호수당전력사용량, dtype: float64

In [10]:
gwangju_g

시군구
북구     14151698
광산구    13740975
서구      7963358
남구      7735608
동구      3608790
Name: 전력사용량, dtype: int64

In [11]:
geo_path = 'data/hangjeongdong_광주광역시.geojson'
#json 파일을 geo_path 변수에 저장
geo_str = json.load(open(geo_path, encoding='utf-8'))
#json 파일을 load 하기 위해 위에서 json파일을 넣어둔 변수를 불러오고 언어를 utf-8 로 변경 후 geo_str 에 저장
geo_str
#json 파일 정보 보기

{'type': 'FeatureCollection',
 'name': 'temp',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 913,
    'adm_nm': '광주광역시 동구 충장동',
    'adm_cd': '2401051',
    'adm_cd2': '2911052500',
    'sgg': '29110',
    'sido': '29',
    'sidonm': '광주광역시',
    'sggnm': '동구'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[126.91920507407151, 35.15325407909361],
       [126.91917063299555, 35.153225637406486],
       [126.91971301742304, 35.15281850488077],
       [126.92115623098215, 35.15175773638071],
       [126.92216781183716, 35.15119298876562],
       [126.92392414654104, 35.149780566865914],
       [126.92504723757074, 35.14884789846158],
       [126.92507819645617, 35.14882218712237],
       [126.9249284087379, 35.14867927537984],
       [126.92433928111625, 35.14811725029431],
       [126.92223184607597, 35.148363067205885],
       [126.92163654563642, 35.14825873197957],
 

In [12]:
gwangju.dropna(inplace=True)
gwangju.reset_index(inplace=True,drop=True)
gwangju.head()

시도  시군구  법정동    년도  월  전체호수  전력사용량  호수당전력사용량
0  광주광역시  광산구  고룡동  2019  1     0      0       0.0
1  광주광역시  광산구  고룡동  2019  2     0      0       0.0
2  광주광역시  광산구  고룡동  2019  3     0      0       0.0
3  광주광역시  광산구  고룡동  2019  4     0      0       0.0
4  광주광역시  광산구  고룡동  2019  5     0      0       0.0

In [13]:
map = folium.Map((35.1557,126.8354),zoom_start=8)
#map(위도,경도) 입력, zoom_start = 얼마나 확대할건지
jsonfile = open('data/hangjeongdong_광주광역시.geojson','r',encoding='utf8')
jsondata = json.load(jsonfile)
jsondata_gwangju = {'type':'FeatureCollection'}
#jsondata type 를 찾아서 jsondata_gwangju 에 저장
jsondata_pick = []
#pick 빈 리스트 생성
jsondata_dong = []
#dong 빈 리스트 생성
for idx in jsondata['features']:
#jsondata에 features 값들을 idx 에 저장
    if idx ['properties']['sidonm'] == "광주광역시":
    #만약 idx값에 ['properties']['sidonm'] 값이 광주 광역시일떄
        dong = idx['properties']['sggnm']
        jsondata_dong.append(dong)
        #dong리스트에 값을 추가
        idx['id'] = dong
        jsondata_pick.append(idx)
        #pick리스트에 값 추가
jsondata_gwangju['features'] =jsondata_pick
#추가된 pick 데이터를 jsondata_gwangju['features'] 에 저장

In [14]:
map = folium.Map((35.1557,126.8354),zoom_start=10)
folium.Choropleth(geo_data=jsondata_gwangju,
                 data=gwangju_ggh,
                  columns=['시군구','전력사용량'],
                  key_on='feature.id',
                 legend_name='광주 호수당전력사용량 ',
                 bins=10,fill_color='YlOrRd').add_to(map)
map
#geo_data = 사용할 jsondata 파일
#columns = 사용할 컬럼
#key_on = json 파일과 데이터 프레임을 연결해주는 역활 / id,name 값으로 사용이 가능
#사용할때 감싸고 있는 컬럼표현 방식을 feature: {id} -> feature.id << 이런식으로 사용가능

In [15]:
# 기간별로 나눠진 가구평균 월사용량 데이터 통합
df_1 = pd.concat([pd.read_excel('data/광주광역시 가구평균 월사용량(19_20)/가구 평균 월별 전력사용량_2019.01~06.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(19_20)/가구 평균 월별 전력사용량_2019.07~12.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(19_20)/가구 평균 월별 전력사용량_2020.01~06.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(19_20)/가구 평균 월별 전력사용량_2020.07~12.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(21)/가구 평균 월별 전력사용량_2021.01~06.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(21)/가구 평균 월별 전력사용량_2021.07~12.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(21)/가구 평균 월별 전력사용량_2022.01~06.xls'),
           pd.read_excel('data/광주광역시 가구평균 월사용량(21)/가구 평균 월별 전력사용량_2022.07~12.xls')],
         ignore_index=True)

In [16]:
# object형식인 '대상가구수(호)'컬럼의 ','를 제거해주고 int로 자료형을 변환
df_1['대상가구수(호)'].replace(',', '',regex=True, inplace = True)
df_1['대상가구수(호)'] = df_1['대상가구수(호)'].astype('int64')

In [17]:
# 전력사용량 컬럼을 가구호수와 가구당 평균전력을 곱해서 추가해준다
df_1['전력 사용량(kWh)'] = df_1['대상가구수(호)']*df_1['가구당 평균 전력 사용량(kWh)']

In [18]:
df_2 = df_1.groupby('시군구')['가구당 평균 전력 사용량(kWh)'].mean()

In [19]:
map = folium.Map((35.1557,126.8354),zoom_start=8)
jsonfile = open('data/hangjeongdong_광주광역시.geojson','r',encoding='utf8')
jsondata = json.load(jsonfile)
jsondata_gwangju = {'type':'FeatureCollection'}
jsondata_pick = []
jsondata_dong = []
for idx in jsondata['features']:
    if idx ['properties']['sidonm'] == "광주광역시":
        dong = idx['properties']['sggnm']
        jsondata_dong.append(dong)
        idx['id'] = dong
        jsondata_pick.append(idx)
jsondata_gwangju['features'] =jsondata_pick


In [20]:
map = folium.Map((35.1557,126.8354),zoom_start=10)
folium.Choropleth(geo_data=jsondata_gwangju,
                 data=df_2,
                  columns=['시군구','가구당 평균 전력 사용량(kWh)'],
                  key_on='feature.id',
                 legend_name='광주 가구당 평균 전력 사용량 ',
                 bins=10,fill_color='YlOrRd').add_to(map)
map

In [21]:
map = folium.Map((35.1557,126.8354),zoom_start=8)
jsonfile = open('data/hangjeongdong_광주광역시.geojson','r',encoding='utf8')
jsondata = json.load(jsonfile)
jsondata_gwangju = {'type':'FeatureCollection'}
jsondata_pick = []
jsondata_dong = []
for idx in jsondata['features']:
    if idx ['properties']['sidonm'] == "광주광역시":
        dong = idx['properties']['sggnm']
        jsondata_dong.append(dong)
        idx['id'] = dong
        jsondata_pick.append(idx)
jsondata_gwangju['features'] =jsondata_pick


In [22]:
map = folium.Map((35.1557,126.8354),zoom_start=10)
folium.Choropleth(geo_data=jsondata_gwangju,
                 data=gwangju_g1,
                  columns=['시군구','전력사용량'],
                  key_on='feature.id',
                 legend_name='광주 전력사용량 ',
                 bins=10,fill_color='YlOrRd').add_to(map)
map